In [ ]:
# General utilities
import os
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from dotenv import load_dotenv
import glob

# NetCDF / data handling
import xarray as xr

# Plotting
import matplotlib.pyplot as plt

# Copernicus Marine Toolbox
from copernicusmarine import login, subset

# Time series modeling
from statsmodels.tsa.arima.model import ARIMA

# OpenDrift simulation
from opendrift.models.oceandrift import OceanDrift
from opendrift.readers.reader_netCDF_CF_generic import Reader

In [ ]:
load_dotenv()

# Fetch credentials
username = os.getenv("CMDS_USERNAME")
password = os.getenv("CMDS_PASSWORD")

if not username or not password:
    raise ValueError("CMDS_USERNAME or CMDS_PASSWORD not set in .env")

# Authenticate with Copernicus Marine
login(username=username, password=password)

In [ ]:
def download_cmems(year, output_dir,
                   min_lon=160, max_lon=180, min_lat=-77, max_lat=-72,
                   start_depth=0, end_depth=1):
    import os
    from datetime import datetime
    from copernicusmarine import subset
    import xarray as xr

    os.makedirs(output_dir, exist_ok=True)
    output_file = f"copernicus_currents_{year}.nc"
    output_path = os.path.join(output_dir, output_file)

    if os.path.exists(output_path):
        print(f"✅ Already exists: {output_path}")
        return

    # Dataset selection
    if year <= 2020:
        dataset_id = "cmems_mod_glo_phy_my_0.083deg_P1D-m"
    else:
        dataset_id = "cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m"
    variables = ['uo', 'vo']

    if year == 2025:
        # Special case: only download January 2025
        try:
            print(f"⬇️ Downloading January {year} only...")
            subset(
                dataset_id=dataset_id,
                minimum_longitude=min_lon,
                maximum_longitude=max_lon,
                minimum_latitude=min_lat,
                maximum_latitude=max_lat,
                minimum_depth=start_depth,
                maximum_depth=end_depth,
                start_datetime=f"{year}-01-01 00:00:00",
                end_datetime=f"{year}-01-31 23:59:59",
                variables=variables,
                output_directory=output_dir,
                output_filename=output_file,
                netcdf3_compatible=True
            )
            print(f"✅ Saved: {output_path}")
        except Exception as e:
            print(f"❌ Failed for January 2025: {e}")
        return

    # Normal case: download both December and January
    dec_file = os.path.join(output_dir, f"tmp_{year}_dec.nc")
    jan_file = os.path.join(output_dir, f"tmp_{year}_jan.nc")

    success_dec = False
    success_jan = False

    try:
        print(f"⬇️ Downloading December {year}...")
        subset(
            dataset_id=dataset_id,
            minimum_longitude=min_lon,
            maximum_longitude=max_lon,
            minimum_latitude=min_lat,
            maximum_latitude=max_lat,
            minimum_depth=start_depth,
            maximum_depth=end_depth,
            start_datetime=f"{year}-12-01 00:00:00",
            end_datetime=f"{year}-12-31 23:59:59",
            variables=variables,
            output_directory=output_dir,
            output_filename=os.path.basename(dec_file),
            netcdf3_compatible=True
        )
        success_dec = os.path.exists(dec_file)
    except Exception as e:
        print(f"⚠️ December {year} failed: {e}")

    try:
        print(f"⬇️ Downloading January {year + 1}...")
        subset(
            dataset_id=dataset_id,
            minimum_longitude=min_lon,
            maximum_longitude=max_lon,
            minimum_latitude=min_lat,
            maximum_latitude=max_lat,
            minimum_depth=start_depth,
            maximum_depth=end_depth,
            start_datetime=f"{year + 1}-01-01 00:00:00",
            end_datetime=f"{year + 1}-01-31 23:59:59",
            variables=variables,
            output_directory=output_dir,
            output_filename=os.path.basename(jan_file),
            netcdf3_compatible=True
        )
        success_jan = os.path.exists(jan_file)
    except Exception as e:
        print(f"⚠️ January {year + 1} failed: {e}")

    if success_dec and success_jan:
        try:
            ds_dec = xr.open_dataset(dec_file)
            ds_jan = xr.open_dataset(jan_file)
            ds_combined = xr.concat([ds_dec, ds_jan], dim="time")
            ds_combined.to_netcdf(output_path)
            print(f"✅ Saved: {output_path}")
        except Exception as e:
            print(f"❌ Failed to merge/save for {year}: {e}")
    else:
        print(f"⚠️ Skipping save for {year}: missing one of the months")

    # Cleanup
    for f in [dec_file, jan_file]:
        if os.path.exists(f):
            os.remove(f)


In [ ]:
for year in range(2013, 2026):
    download_cmems(year, output_dir="data")


In [ ]:
dj_uo_series = {}
dj_vo_series = {}
lat = None
lon = None

for year in range(2013, 2025):  # up to 2024 inclusive
    file_dec = f"data/copernicus_currents_{year}.nc"
    file_jan = f"data/copernicus_currents_{year + 1}.nc"

    if not (os.path.exists(file_dec) and os.path.exists(file_jan)):
        print(f"⚠️ Skipping year {year}: missing Dec or Jan file.")
        continue

    try:
        ds_dec = xr.open_dataset(file_dec)
        ds_dec = ds_dec.where(ds_dec['time'].dt.month == 12, drop=True)

        ds_jan = xr.open_dataset(file_jan)
        ds_jan = ds_jan.where(ds_jan['time'].dt.month == 1, drop=True)

        ds_dj = xr.concat([ds_dec, ds_jan], dim="time")
    except Exception as e:
        print(f"⚠️ Failed to open/filter DJ data for {year}: {e}")
        continue

    if lat is None:
        lat = ds_dj.latitude.values
        lon = ds_dj.longitude.values

    for i in range(len(lat)):
        for j in range(len(lon)):
            try:
                key = (i, j)
                u_vals = ds_dj['uo'][:, i, j].values
                v_vals = ds_dj['vo'][:, i, j].values

                if np.isnan(u_vals).all() or np.isnan(v_vals).all():
                    continue

                dj_uo_series.setdefault(key, []).append(np.nanmean(u_vals))
                dj_vo_series.setdefault(key, []).append(np.nanmean(v_vals))
            except Exception:
                continue


In [ ]:
# Load December 2024 and January 2025 actual Copernicus data
reader_dec = Reader('data/copernicus_currents_2024.nc')
reader_jan = Reader('data/copernicus_currents_2025.nc')

# Initialize OceanDrift model
o = OceanDrift(loglevel=20)
o.add_reader([reader_dec, reader_jan])  # add both readers

# Nurdle properties
nurdle_diameter = 0.0035  # meters
nurdle_density = 980      # kg/m^3
total_mass = 740000       # kg

particle_volume = (4/3) * np.pi * (nurdle_diameter/2)**3
particle_mass = nurdle_density * particle_volume
simulated_particles = 2000

print(f"Simulating {simulated_particles} particles with total mass {total_mass} kg")

# Seed region around central Ross Sea
seed_lons = np.random.uniform(169.5, 170.5, simulated_particles)
seed_lats = np.random.uniform(-75.5, -74.5, simulated_particles)

# Seed particles on December 1, 2024
o.seed_elements(
    lon=seed_lons,
    lat=seed_lats,
    time=datetime(2024, 12, 1),
)

# Run simulation through January 2025
o.run(end_time=datetime(2025, 1, 31), time_step=3600, time_step_output=86400)


In [ ]:
o.plot(fast=True, legend=True, filename=None)

In [ ]:
o.animation(
    filename="nurdle_dispersion_dec_2024_jan_2025.mp4",
    show_elements=True,
    show_trajectory=True,
    marker_size=2,
    dpi=150,
    fast=True,
    show_time=True,
    show_legend=True
)

## WORK IN PROGRESS BELOW!!!

### ARIMA model for December-January currents

In [ ]:
# Load and combine all available NetCDF files from 2013–2025
data_dir = "data"
all_files = sorted(glob.glob(os.path.join(data_dir, "copernicus_currents_*.nc")))
ds_list = [xr.open_dataset(f) for f in all_files if os.path.exists(f)]
combined = xr.concat(ds_list, dim="time")
uo, vo = combined["uo"], combined["vo"]

# Forecast shape: (62, lat, lon)
forecast_days = 62
lat_len, lon_len = uo.shape[1], uo.shape[2]
forecast_arima_dec_2025_jan_2026 = {"uo": np.zeros((forecast_days, lat_len, lon_len)),
                                    "vo": np.zeros((forecast_days, lat_len, lon_len))}

def arima_forecast_3d(component):
    result = np.full((forecast_days, lat_len, lon_len), np.nan)
    for i in range(lat_len):
        for j in range(lon_len):
            series = component[:, i, j].values
            if np.any(np.isnan(series)):
                continue
            try:
                model = ARIMA(series, order=(1, 0, 0)).fit()
                forecast = model.forecast(steps=forecast_days)
                result[:, i, j] = forecast
            except:
                continue
    return result

forecast_arima_dec_2025_jan_2026["uo"] = arima_forecast_3d(uo)
forecast_arima_dec_2025_jan_2026["vo"] = arima_forecast_3d(vo)

time = pd.date_range("2025-12-01", periods=62)
lat = np.linspace(-75, -70, forecast_arima_dec_2025_jan_2026["uo"].shape[1])
lon = np.linspace(160, 170, forecast_arima_dec_2025_jan_2026["uo"].shape[2])

# Save ARIMA forecast
ds_arima = xr.Dataset(
    {
        "uo": (["time", "lat", "lon"], forecast_arima_dec_2025_jan_2026["uo"]),
        "vo": (["time", "lat", "lon"], forecast_arima_dec_2025_jan_2026["vo"]),
    },
    coords={"time": time, "lat": lat, "lon": lon}
)
ds_arima.to_netcdf("data/forecast_arima_dec_2025_jan_2026.nc")



In [ ]:
# Load December 2024 and January 2025 actual Copernicus data
reader = Reader('data/forecast_arima_dec_2025_jan_2026.nc')

# Initialize OceanDrift model
o = OceanDrift(loglevel=20)
o.add_reader([reader_dec, reader_jan])  # add both readers

# Nurdle properties
nurdle_diameter = 0.0035  # meters
nurdle_density = 980      # kg/m^3
total_mass = 740000       # kg

particle_volume = (4/3) * np.pi * (nurdle_diameter/2)**3
particle_mass = nurdle_density * particle_volume
simulated_particles = 2000

print(f"Simulating {simulated_particles} particles with total mass {total_mass} kg")

# Seed region around central Ross Sea
seed_lons = np.random.uniform(169.5, 170.5, simulated_particles)
seed_lats = np.random.uniform(-75.5, -74.5, simulated_particles)

# Seed particles on December 1, 2025
o.seed_elements(
    lon=seed_lons,
    lat=seed_lats,
    time=datetime(2025, 12, 1),
)

# Run simulation through January 2026
o.run(end_time=datetime(2026, 1, 31), time_step=3600, time_step_output=86400)


In [ ]:
print(reader)

In [ ]:
o.plot(fast=True, legend=True, filename=None)